Establecemos una ``semilla`` que nos permitirá hacer que los experimentos sean reproducibles.

In [1]:
#semilla
seed = 42

Realizamos todas las `importaciones` necesarias.

In [2]:
#Importamos las librerias
from sklearn.model_selection import train_test_split
from sklearn import set_config
import matplotlib.pyplot as plt

import pandas as pd

import os
from PIL import Image

# 1.Carga de datos

In [3]:
image_dir = 'sin_procesar'

images = []

for filename in os.listdir(image_dir):
    if filename.endswith('.jpg'): 
        img_path = os.path.join(image_dir, filename)
        img = Image.open(img_path)
        images.append(img)

# Print the number of images loaded
print(f'Loaded {len(images)} images.')
#indica el nombre de la imagen en la posicion 2
print(images[2].filename)

Loaded 70 images.
sin_procesar\11.jpg


In [4]:
set_config(transform_output="pandas")
df = pd.read_csv('Hito2Datos.csv', delimiter=';')

In [5]:
# Definir las etiquetas
etiquetas = ['URL imagen', 'ID', 'Inundado', 'transitable', 'formarURL', 'InundadoJm', 'TransitableJm', 'InundadoA', 'TransitableA', 'InundadoI', 'TransitableI', 'InundadoJ', 'TransitableJ', 'InundadoC', 'TransitableC']

# Rellenar las columnas 'Inundado' y 'transitable' con valores de otras columnas
df['Inundado'] = df[['InundadoJm', 'InundadoA', 'InundadoI', 'InundadoJ', 'InundadoC']].mode(axis=1)[0]
df['transitable'] = df[['TransitableJm', 'TransitableA', 'TransitableI', 'TransitableJ', 'TransitableC']].mode(axis=1)[0]

# Mostrar el dataframe actualizado
print(df.head())

  URL imagen  ID Inundado transitable  \
0     Imagen   1       Si          No   
1     Imagen   2       Si          No   
2     Imagen   3       Si          No   
3     Imagen   4       Si          No   
4     Imagen   5       Si          No   

                                           formarURL InundadoJm TransitableJm  \
0  https://raw.githubusercontent.com/alenavarroxp...         Si            No   
1  https://raw.githubusercontent.com/alenavarroxp...         Si            No   
2  https://raw.githubusercontent.com/alenavarroxp...         No            Si   
3  https://raw.githubusercontent.com/alenavarroxp...         Si            No   
4  https://raw.githubusercontent.com/alenavarroxp...         Si            No   

  InundadoA TransitableA InundadoI TransitableI InundadoJ TransitableJ  \
0        Si           No        Si           No        Si           No   
1        Si           No        Si           No        Si           No   
2        Si           No        Si          

Comprobando que se ha cargado correctamente:

In [6]:
df.sample(5, random_state=seed)

,URL imagen,ID,Inundado,transitable,formarURL,InundadoJm,TransitableJm,InundadoA,TransitableA,InundadoI,TransitableI,InundadoJ,TransitableJ,InundadoC,TransitableC
22,Imagen,23,Si,No,https://raw.githubusercontent.com/alenavarroxp...,Si,No,Si,Si,Si,No,Si,No,Si,No
0,Imagen,1,Si,No,https://raw.githubusercontent.com/alenavarroxp...,Si,No,Si,No,Si,No,Si,No,Si,No
49,Imagen,50,No,No,https://raw.githubusercontent.com/alenavarroxp...,No,No,No,No,No,No,No,No,No,No
4,Imagen,5,Si,No,https://raw.githubusercontent.com/alenavarroxp...,Si,No,Si,Si,Si,No,Si,No,Si,No
54,Imagen,55,No,Si,https://raw.githubusercontent.com/alenavarroxp...,No,Si,No,Si,No,Si,No,Si,No,Si


A continuación, creamos ``etiquetas`` para separar la variable objetivo del resto de variables.

In [7]:
target = ["transitable", "Inundado"]
columns = target
axis = "columns"

arguments = {"columns": columns, "axis": axis}
X, y = df.drop(**arguments), df[target]

Vamos a comprobar que se ha separado correctamente. Comenzamos con las `variables predictoras`:

In [8]:
X.sample(5, random_state=seed)

,URL imagen,ID,formarURL,InundadoJm,TransitableJm,InundadoA,TransitableA,InundadoI,TransitableI,InundadoJ,TransitableJ,InundadoC,TransitableC
22,Imagen,23,https://raw.githubusercontent.com/alenavarroxp...,Si,No,Si,Si,Si,No,Si,No,Si,No
0,Imagen,1,https://raw.githubusercontent.com/alenavarroxp...,Si,No,Si,No,Si,No,Si,No,Si,No
49,Imagen,50,https://raw.githubusercontent.com/alenavarroxp...,No,No,No,No,No,No,No,No,No,No
4,Imagen,5,https://raw.githubusercontent.com/alenavarroxp...,Si,No,Si,Si,Si,No,Si,No,Si,No
54,Imagen,55,https://raw.githubusercontent.com/alenavarroxp...,No,Si,No,Si,No,Si,No,Si,No,Si


Y continuamos con la `variable clase`:

In [9]:
y.sample(5, random_state=seed)

,transitable,Inundado
22,No,Si
0,No,Si
49,No,No
4,No,Si
54,Si,No


Por último, `dividimos` el conjunto de datos en entrenamiento y prueba mediante un *holdout* estratificado:

In [10]:
test_size = 0.3

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed, stratify=y)

In [11]:
# Obtener los nombres de las imágenes en X_train
image_id = X_train['ID'].tolist()
# Imprimir los nombres de las imágenes
print(image_id)
image_names = []
# recorremos la lista de nombres de las images y creamos una lista con las imagenes de image_names
for i in image_id:
    image_names.append(str(i)+'.jpg')
    
images_train = []
for img in images:
    if img.filename.split("\\")[-1] in image_names:
        images_train.append(img)
images_train[0].filename

[68, 21, 17, 36, 10, 31, 60, 48, 69, 11, 39, 52, 62, 57, 46, 5, 13, 44, 65, 32, 6, 24, 42, 30, 22, 54, 49, 40, 66, 64, 14, 3, 29, 8, 27, 20, 26, 4, 70, 33, 28, 55, 37, 61, 47, 35, 59, 18, 7]


'sin_procesar\\10.jpg'

Y nos aseguramos que se ha realizado adecuadamente. Comenzamos con el conjunto de datos de entrenamiento:

In [12]:
X_train.sample(5, random_state=seed)

,URL imagen,ID,formarURL,InundadoJm,TransitableJm,InundadoA,TransitableA,InundadoI,TransitableI,InundadoJ,TransitableJ,InundadoC,TransitableC
56,Imagen,57,https://raw.githubusercontent.com/alenavarroxp...,No,Si,No,Si,No,Si,No,Si,No,Si
34,Imagen,35,https://raw.githubusercontent.com/alenavarroxp...,No,No,No,No,No,Si,No,Si,No,Si
17,Imagen,18,https://raw.githubusercontent.com/alenavarroxp...,Si,No,Si,Si,Si,Si,Si,No,Si,No
46,Imagen,47,https://raw.githubusercontent.com/alenavarroxp...,No,No,No,No,No,Si,No,No,No,No
43,Imagen,44,https://raw.githubusercontent.com/alenavarroxp...,No,No,No,No,No,No,No,No,No,No


In [13]:
y_train.sample(5, random_state=seed)

,transitable,Inundado
56,Si,No
34,Si,No
17,No,Si
46,No,No
43,No,No


Y finalizamos con el conjunto de datos de prueba:

In [14]:
X_test.sample(5, random_state=seed)

,URL imagen,ID,formarURL,InundadoJm,TransitableJm,InundadoA,TransitableA,InundadoI,TransitableI,InundadoJ,TransitableJ,InundadoC,TransitableC
14,Imagen,15,https://raw.githubusercontent.com/alenavarroxp...,Si,No,Si,No,Si,No,Si,No,Si,No
1,Imagen,2,https://raw.githubusercontent.com/alenavarroxp...,Si,No,Si,No,Si,No,Si,No,Si,No
49,Imagen,50,https://raw.githubusercontent.com/alenavarroxp...,No,No,No,No,No,No,No,No,No,No
40,Imagen,41,https://raw.githubusercontent.com/alenavarroxp...,No,No,No,No,No,No,No,No,No,No
50,Imagen,51,https://raw.githubusercontent.com/alenavarroxp...,No,Si,No,Si,No,Si,No,Si,No,Si


In [15]:
y_test.sample(5, random_state=seed)

,transitable,Inundado
14,No,Si
1,No,Si
49,No,No
40,No,No
50,Si,No


Se borran las etiquetas que no aportan informacion

In [16]:
etiquetas=['URL imagen,formarURL,InundadoJm,TransitableJm,InundadoA,TransitableA,InundadoI,TransitableI,InundadoJ,TransitableJ,InundadoC,TransitableC'] 

In [17]:
print(etiquetas)

['URL imagen,formarURL,InundadoJm,TransitableJm,InundadoA,TransitableA,InundadoI,TransitableI,InundadoJ,TransitableJ,InundadoC,TransitableC']


In [18]:
print(X_train.columns)

Index(['URL imagen', 'ID', 'formarURL', 'InundadoJm', 'TransitableJm',
       'InundadoA', 'TransitableA', 'InundadoI', 'TransitableI', 'InundadoJ',
       'TransitableJ', 'InundadoC', 'TransitableC'],
      dtype='object')


# ELIMINACIÓN DE VARIABLES

vamos a `eliminar` las variables que no aportan información con la ayuda de un `pipeline`.

In [19]:
def drop_columns(X, etiquetas):
    # Convertir el string de 'etiquetas' en una lista de columnas
    etiquetas = etiquetas[0].split(',')
    
    # Eliminar las columnas que existen en el dataframe
    return X.drop(columns=etiquetas)

In [20]:
X_train=drop_columns(X_train, etiquetas)
X_train.sample(5, random_state=seed)

,ID
56,57
34,35
17,18
46,47
43,44


In [21]:
X_test=drop_columns(X_test, etiquetas)
X_test.sample(5, random_state=seed)

,ID
14,15
1,2
49,50
40,41
50,51


# ENCODER y_train e y_test

In [22]:
y_train = y_train.applymap(lambda x: 1 if x == "Si" else (0 if x == "No" else x))
y_train.head()

C:\Users\Juanmi\AppData\Local\Temp\ipykernel_24060\1745860845.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  y_train = y_train.applymap(lambda x: 1 if x == "Si" else (0 if x == "No" else x))


,transitable,Inundado
67,1,0
20,0,1
16,1,1
35,0,0
9,0,1


In [23]:
y_test = y_test.applymap(lambda x: 1 if x == "Si" else (0 if x == "No" else x))
y_test.head()

C:\Users\Juanmi\AppData\Local\Temp\ipykernel_24060\3414226292.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  y_test = y_test.applymap(lambda x: 1 if x == "Si" else (0 if x == "No" else x))


,transitable,Inundado
14,0,1
40,0,0
55,1,0
24,0,1
15,0,1


# TRANSFORMACION DE LAS IMÁGENES

In [24]:
# Convertir imágenes a escala de grises y rotarlas
augmented_images = []

for img in images_train:
    # Convertir a escala de grises
    gray_img = img.convert('L')
    gray_img.filename = "0_"+img.filename.split("\\")[-1]
    
    # Rotar la imagen 90º, 180º y 270º
    rotated_90 = gray_img.rotate(90)
    rotated_180 = gray_img.rotate(180)
    rotated_270 = gray_img.rotate(270)
    rotated_90.filename = "90_"+img.filename.split("\\")[-1]
    rotated_180.filename = "180_"+img.filename.split("\\")[-1]
    rotated_270.filename = "270_"+img.filename.split("\\")[-1]
    
    # Añadir las imágenes originales y transformadas a la lista con filenames
    augmented_images.extend([gray_img, rotated_90, rotated_180, rotated_270])

# Imprimir el número de imágenes aumentadas
print(f'Número total de imágenes aumentadas: {len(augmented_images)}')
augmented_images[2].filename

Número total de imágenes aumentadas: 196


'180_10.jpg'

In [25]:
output_dir = 'trainprocesados'
os.makedirs(output_dir, exist_ok=True)

for img in augmented_images:
    img.save(os.path.join(output_dir, img.filename.split("\\")[-1]))

print(f'Se han guardado {len(augmented_images)} imágenes en la carpeta {output_dir}.')

Se han guardado 196 imágenes en la carpeta trainprocesados.


In [26]:
# Crear la carpeta MetaData si no existe
os.makedirs('MetaData', exist_ok=True)

# Juntar X_train e y_train
train_data = pd.concat([X_train, y_train], axis=1)
train_data.to_csv('MetaData/train_data.csv', index=False)

# Juntar X_test e y_test
test_data = pd.concat([X_test, y_test], axis=1)
test_data.to_csv('MetaData/test_data.csv', index=False)

print('Los archivos CSV se han guardado en la carpeta MetaData.')

Los archivos CSV se han guardado en la carpeta MetaData.
